# Implementing Batch Normalisation with Keras

## Load the dataset and create training set, test set and validation set

In [1]:
# import the lib
import tensorflow as tf
from tensorflow import keras

# load the dataset
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# scale 
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# split for validation set
X_valid, X_train = X_train_full[: 5000], X_train_full[5000: ]
y_valid, y_train = y_train_full[: 5000], y_train_full[5000: ]

2021-11-07 08:01:41.807574: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-07 08:01:41.808623: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## create the model with BN layers

In [2]:
# import the lib
import numpy as np

# set the random seed
np.random.seed(42)
tf.random.set_seed(42)

# create the model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"), 
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

2021-10-15 13:29:00.155920: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-15 13:29:00.156064: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-15 13:29:00.156173: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-8E5U3B3): /proc/driver/nvidia/version does not exist
2021-10-15 13:29:00.167935: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

## check the parameters of the first BN layer

In [4]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

## add BN layers before activation functions

In [9]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])

In [10]:
# compile the model
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.SGD(learning_rate=1e-3),
             metrics=["accuracy"])

In [11]:
# history
history = model.fit(X_train, y_train, epochs=10,
                   validation_data=(X_valid, y_valid))

2021-10-15 19:18:33.271764: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.9161 - accuracy: 0.6988 - val_loss: 0.6329 - val_accuracy: 0.7954
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6370 - accuracy: 0.7866 - val_loss: 0.5427 - val_accuracy: 0.8192
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5747 - accuracy: 0.8044 - val_loss: 0.5043 - val_accuracy: 0.8324
Epoch 4/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.5388 - accuracy: 0.8167 - val_loss: 0.4796 - val_accuracy: 0.8394
Epoch 5/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.5142 - accuracy: 0.8221 - val_loss: 0.4631 - val_accuracy: 0.8418
Epoch 6/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4986 - accuracy: 0.8271 - val_loss: 0.4502 - val_accuracy: 0.8482
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4865 - accuracy: 0.8314 - val_loss: 0.4394 - val_accuracy:

## Gradient Clipping

In [12]:
# define the optimizer
optimizer = keras.optimizers.SGD(clipvalue=1.0, learning_rate=1e-3)

# compile the model with the defined optimizer
model.compile(loss="mse",
             optimizer=optimizer,
             metrics=["accuracy"])

## Reusing Pre-Trained Layers

### split the MNIST dataset into dataset A and dataset B 

In [38]:
# define a function for splitting data
def split_dataset(X, y):
    
    # create labels y_A for dataset A
    ## find the condition that the label corresponds to sandals or shirts
    y_5_or_6 = (y == 5) | (y == 6)
    
    ## set all the non-sandal and non-shirt labels together for dataset A
    y_A = y[~y_5_or_6]
    
    ## the original labels 7, 8, 9 should be set to 5, 6, 7 respectively
    y_A[y_A > 6] -= 2
    
    # create labels y_B for dataset B
    y_B = (y[y_5_or_6] == 6).astype(np.float32)
    
    return ((X[~y_5_or_6], y_A), 
            (X[y_5_or_6], y_B))

# split the training set into A and B
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)

# split the validation set into A and B
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)

# split the test set into A and B
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)

# assign the first 200 instances of training set B for the task
X_train_B = X_train_B[: 200]
y_train_B = y_train_B[: 200]

In [27]:
X_train_A.shape

(43986, 28, 28)

In [28]:
X_train_B.shape

(200, 28, 28)

### create, compile and history the model A

In [30]:
# import the lib
import numpy as np
import tensorflow as tf
from tensorflow import keras

# set the random seed
np.random.seed(42)
tf.random.set_seed(42)

# initialise the model A 
model_A = keras.models.Sequential()

# add layers on model A
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

# compile the model
model_A.compile(loss="sparse_categorical_crossentropy",
               optimizer=keras.optimizers.SGD(learning_rate=1e-3),
               metrics=["accuracy"])

# history fit
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                     validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 4s 2ms/step - loss: 0.5422 - accuracy: 0.8223 - val_loss: 0.3624 - val_accuracy: 0.8777
Epoch 2/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3374 - accuracy: 0.8833 - val_loss: 0.3084 - val_accuracy: 0.8956
Epoch 3/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3032 - accuracy: 0.8951 - val_loss: 0.2824 - val_accuracy: 0.9033
Epoch 4/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2848 - accuracy: 0.9027 - val_loss: 0.2703 - val_accuracy: 0.9098
Epoch 5/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2723 - accuracy: 0.9058 - val_loss: 0.2612 - val_accuracy: 0.9111
Epoch 6/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2632 - accuracy: 0.9087 - val_loss: 0.2579 - val_accuracy: 0.9136
Epoch 7/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2556 - accuracy: 0.9119 - val_loss: 0.2533 - val_accuracy:

### save the model A

In [31]:
model_A.save("my_model_A.h5")

### create, compile, and history-fit model B

In [34]:
# initialise model B
model_B = keras.models.Sequential()

# add input layer to model B
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))

# add hidden layer to model B
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
    
# add output layer
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

# compile the model B
model_B.compile(loss="binary_crossentropy",
               optimizer=keras.optimizers.SGD(learning_rate=1e-3),
               metrics=["accuracy"])

# hitory fit
history_B = model_B.fit(X_train_B, y_train_B, epochs=20,
                       validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 28ms/step - loss: 0.7163 - accuracy: 0.5300 - val_loss: 0.5416 - val_accuracy: 0.7343
Epoch 2/20
7/7 [==============================] - 0s 12ms/step - loss: 0.4794 - accuracy: 0.8000 - val_loss: 0.4450 - val_accuracy: 0.8580
Epoch 3/20
7/7 [==============================] - 0s 12ms/step - loss: 0.3985 - accuracy: 0.8800 - val_loss: 0.3907 - val_accuracy: 0.8905
Epoch 4/20
7/7 [==============================] - 0s 11ms/step - loss: 0.3490 - accuracy: 0.9100 - val_loss: 0.3515 - val_accuracy: 0.9006
Epoch 5/20
7/7 [==============================] - 0s 12ms/step - loss: 0.3112 - accuracy: 0.9150 - val_loss: 0.3198 - val_accuracy: 0.9128
Epoch 6/20
7/7 [==============================] - 0s 14ms/step - loss: 0.2797 - accuracy: 0.9250 - val_loss: 0.2945 - val_accuracy: 0.9219
Epoch 7/20
7/7 [==============================] - 0s 13ms/step - loss: 0.2546 - accuracy: 0.9400 - val_loss: 0.2748 - val_accuracy: 0.9270
Epoch 8/20
7/7 [===========

In [35]:
# show the summary of B
model_B.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_23 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_24 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_25 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_26 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_27 (Dense)             (None, 1)                

### create  Model B_on_A

In [36]:
# load model A from the h5 file
model_A = keras.models.load_model("my_model_A.h5")

# create the model B_on_A with all the A's layers except from the output layer
model_B_on_A = keras.models.Sequential(model_A.layers[: -1])

# add an output layer to model B_on_A
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# freeze all the reused layers 
for layer in model_B_on_A.layers[: -1]:
    layer.trainable = False

# compile the layer-freezed model B_on_A
model_B_on_A.compile(loss="binary_crossentropy",
                    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                    metrics=["accuracy"])

# 4-epoch history fit
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                          validation_data=(X_valid_B, y_valid_B))

# unfreeze all the reused layers
for layer in model_B_on_A.layers[: -1]:
    layer.trainable = True

# compile the layer-unfreezed model
model_B_on_A.compile(loss="binary_crossentropy",
                    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                    metrics=["accuracy"])

# 16-epoch history fit
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                          validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 31ms/step - loss: 1.1216 - accuracy: 0.3850 - val_loss: 1.0541 - val_accuracy: 0.4493
Epoch 2/4
7/7 [==============================] - 0s 12ms/step - loss: 1.0192 - accuracy: 0.4400 - val_loss: 0.9636 - val_accuracy: 0.5030
Epoch 3/4
7/7 [==============================] - 0s 12ms/step - loss: 0.9287 - accuracy: 0.5100 - val_loss: 0.8840 - val_accuracy: 0.5335
Epoch 4/4
7/7 [==============================] - 0s 11ms/step - loss: 0.8491 - accuracy: 0.5650 - val_loss: 0.8113 - val_accuracy: 0.5832
Epoch 1/16
7/7 [==============================] - 0s 28ms/step - loss: 0.5939 - accuracy: 0.7050 - val_loss: 0.4185 - val_accuracy: 0.8174
Epoch 2/16
7/7 [==============================] - 0s 12ms/step - loss: 0.3365 - accuracy: 0.8550 - val_loss: 0.2956 - val_accuracy: 0.9047
Epoch 3/16
7/7 [==============================] - 0s 12ms/step - loss: 0.2428 - accuracy: 0.9300 - val_loss: 0.2347 - val_accuracy: 0.9331
Epoch 4/16
7/7 [===============

In [39]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.1439 - accuracy: 0.9705


[0.14390385150909424, 0.9704999923706055]

In [40]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0722 - accuracy: 0.9910


[0.0721736028790474, 0.9909999966621399]

## Regularization with Dropout

In [2]:
# load the data
# import the lib
import tensorflow as tf
from tensorflow import keras

# load the dataset
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# scale 
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# split for validation set
X_valid, X_train = X_train_full[: 5000], X_train_full[5000: ]
y_valid, y_train = y_train_full[: 5000], y_train_full[5000: ]

In [3]:
# create the model 
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    
    # add Dropout layer in front of all the hidden layers
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    
    # add Dropout layer in front of the output layer
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

# compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

# scale the training instances to mean 0 and standard deviation 1
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

# obtain the history
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                   validation_data=(X_valid_scaled, y_valid))

2021-11-07 08:17:04.415836: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-07 08:17:04.417252: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-07 08:17:04.417477: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-8E5U3B3): /proc/driver/nvidia/version does not exist
2021-11-07 08:17:04.454818: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-07 08:17:06.663784: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLI

Epoch 1/2
1719/1719 [==============================] - 7s 3ms/step - loss: 0.5737 - accuracy: 0.8027 - val_loss: 0.3632 - val_accuracy: 0.8706
Epoch 2/2
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4200 - accuracy: 0.8463 - val_loss: 0.3412 - val_accuracy: 0.8740
